# (15) Load fits

**Motivation**: Device = ```cuda:1``` <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from analysis.eval import sparse_score
from figures.fighelper import *
from vae.train_vae import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Device

In [2]:
device_idx = 1
device = f'cuda:{device_idx}'

from analysis.final import sort_fits, analyze_fits

## Load, make df

In [3]:
fits, fits_st, fits_etc = sort_fits()
len(fits), len(fits_st), len(fits_etc)

(460, 90, 515)

In [4]:
def accept_fn(name):
    archi = name.split('-')[1]
    accept = archi in ['<lin|lin>', '<conv+b|lin>']
    accept = accept and '-b' not in name
    return accept


df = analyze_fits(sorted(filter(accept_fn, fits)), device, 'loss')
print(collections.Counter(df['checkpoint']))

100%|█████████████████████████████████████████| 240/240 [02:29<00:00,  1.60it/s]


Counter({1500: 80, 3000: 80, 1200: 80})

In [5]:
print(collections.Counter(df['seed']))

Counter({1: 48, 2: 48, 3: 48, 4: 48, 5: 48})

In [6]:
print(collections.Counter(df['method']))

Counter({'exact': 120, 'mc': 120})

In [7]:
df['type'].value_counts()

type
categorical    60
gaussian       60
laplace        60
poisson        60
Name: count, dtype: int64

In [8]:
df

,checkpoint,timestamp,dataset,type,enc_type,dec_type,latent_act,n_latents,n_categories,seed,kl_beta,n_params,method,mse,kl,kl_diag,nelbo
0,1500,"2024_05_15,02:05",CIFAR10-PATCHES,categorical,conv,lin,None,1,512.0,1,1.0,1405084,exact,174.093353,6.044831,6.044831,180.138184
1,1500,"2024_05_15,11:37",CIFAR10-PATCHES,categorical,conv,lin,None,1,512.0,2,1.0,1405084,exact,174.858093,6.034148,6.034148,180.892242
2,1500,"2024_05_15,21:14",CIFAR10-PATCHES,categorical,conv,lin,None,1,512.0,3,1.0,1405084,exact,174.635742,6.034076,6.034076,180.669815
3,1500,"2024_05_16,06:50",CIFAR10-PATCHES,categorical,conv,lin,None,1,512.0,4,1.0,1405084,exact,174.558258,6.044322,6.044322,180.602585
4,1500,"2024_05_16,16:27",CIFAR10-PATCHES,categorical,conv,lin,None,1,512.0,5,1.0,1405084,exact,174.533600,6.042276,6.042276,180.575882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1200,"2024_05_14,15:02",MNIST,poisson,lin,lin,None,512,NaN,1,1.0,803328,mc,33.282413,8.503514,0.016608,41.785927
236,1200,"2024_05_14,22:15",MNIST,poisson,lin,lin,None,512,NaN,2,1.0,803328,mc,33.214127,8.516159,0.016633,41.730286
237,1200,"2024_05_14,06:13",MNIST,poisson,lin,lin,None,512,NaN,3,1.0,803328,mc,33.092651,8.510839,0.016623,41.603493
238,1200,"2024_05_14,12:59",MNIST,poisson,lin,lin,None,512,NaN,4,1.0,803328,mc,33.156258,8.499963,0.016601,41.656219


In [9]:
group_cols = ['dataset', 'type', 'enc_type', 'method']
df_avg = df_selected.groupby(by=group_cols).mean(numeric_only=True)
df_avg

NameError: name 'df_selected' is not defined

In [ ]:
df_agg = df_selected.groupby(by=group_cols)['nelbo'].agg(list)

In [ ]:
for index, value in df_agg.items():
    print("Index:", index, "Value:", value)

In [ ]:
data = df_agg[('DOVES', 'poisson', 'conv', 'exact')]

boot = sp_stats.bootstrap(
    data=(data,),
    n_resamples=1000000,
    statistic=np.mean,
    paired=True,
)
print(boot)

In [ ]:
e = boot.standard_error * get_tval(4, 0.95)
(np.mean(data) - e, np.mean(data) + e)

In [ ]:
test = sp_stats.ttest_rel(
    a=df_agg[('DOVES', 'poisson', 'conv', 'exact')],
    b=df_agg[('DOVES', 'poisson', 'conv', 'mc')],
)
test.statistic, test.pvalue

In [ ]:
test = sp_stats.ttest_rel(
    a=df_agg[('CIFAR10-PATCHES', 'poisson', 'lin', 'exact')],
    b=df_agg[('CIFAR10-PATCHES', 'poisson', 'lin', 'mc')],
)
test.statistic, test.pvalue